In [3]:
import os
import time
import sys
import h5py
import re
import tensorflow  as tf
import numpy       as np
import pandas      as pd
from matplotlib    import pyplot as plt 
from os            import path
from tqdm          import tqdm

print("="*50) 
print(" TensorFlow version: {}".format(tf.__version__))
print(" Eager execution: {}".format(tf.executing_eagerly()))

 TensorFlow version: 2.4.1
 Eager execution: True


In [4]:
DATA_PATH      = os.environ['WORKSPACE_PATH']
ReactionType = 'KInel' # KInel, KExch, KDiss
System              = 'N3_NASA'  
Molecules           = ['N2','N2']
Atoms               = ['N','N']

FileName = DATA_PATH+'/../N2_sanjeev.dat'

MinValueTrain = 1.e-15

# Reading and Writing Plato Format

In [35]:
#========================================================================================================
def read_kexcitdata_PLATO(PathToKineticFile):
    print(PathToKineticFile)
    Rates = np.zeros((62,62))
    iIdx = []
    jIdx = []
    ARate = []
    BRate = []
    CRate = []
    
    i2j_dict = {}
    with open(PathToKineticFile) as f:
        index = 0
        for lines in f:
            lines = lines.replace('(',',')
            lines = lines.replace(')',',')
            lines = lines.replace('+N',',')
            linestemp = re.split('N2|N|=|:|,',lines);
            print(lines)
            print(linestemp[2],linestemp[5],linestemp[12])
            iIdxTemp  = int(linestemp[2]);
            jIdxTemp  = int(linestemp[5]);
            ARateTemp = float(linestemp[12]);
            BRateTemp = float(linestemp[13]);
            CRateTemp = float(linestemp[14]);
            
            if iIdx-1 in i2j_dict.keys():
                i2j_dict[iIdx-1].append(jIdx-1)
            else:
                i2j_dict[iIdx-1] = [jIdx-1]
                
            iIdx.append(iIdxTemp)
            jIdx.append(jIdxTemp)
            ARate.append(ARateTemp)
            BRate.append(BRateTemp)
            CRate.append(CRateTemp)
            
            Rates[iIdx-1,jIdx-1] = RateTemp;
            
    f.close()
    
    return Rates, i2j_dict, iIdx, jIdx, ARate, BRate, CRate


#========================================================================================================
def read_kdissdata_PLATO(PathToKineticFile):
    print(PathToKineticFile)
    Rates = np.zeros((NLevels[0],NLevels[0]))
    i2j_dict = {}
    with open(PathToKineticFile) as f:
        for lines in f:
            lines = lines.replace('(',',')
            lines = lines.replace('E','d')
            lines = lines.replace(')',',')
            lines = lines.replace('+',',')
            linestemp = re.split('N2|N|=|:|,',lines);
            iIdx    = int(linestemp[2]);
            RateTemp = float(linestemp[12]);
            
            if iIdx-1 in i2j_dict.keys():
                i2j_dict[iIdx-1].append(jIdx-1)
            else:
                i2j_dict[iIdx-1] = [jIdx-1]
                
            Rates[iIdx-1,jIdx-1] = RateTemp;
    f.close()
    
    return Rates, i2j_dict

#========================================================================================================
def write_predictiondata(ExcitType, KineticFile, csvkinetics, iFlg, Molecules, Atoms, iLevel, jLevelVec, KVec):

    if (iFlg == 0):
        for i in range(len(jLevelVec)): 
            ProcName = Molecules[0] + '(' + str(iLevel+1) + ')+' + Atoms[0] + '=' + Molecules[1] + '(' + str(jLevelVec[i]+1) + ')+' + Atoms[1]
            if (KVec[i] >= MinValueTrain):
                if (ExcitType == 'KInel'):
                    Line     = ProcName + ':%.4e,%.4e,%.4e,5\n' % KVec[i]
                else:
                    Line     = ProcName + ':%.4e,%.4e,%.4e,6\n' % KVec[i]
                csvkinetics.write(Line)

    elif (iFlg == -1):
        print('[SurQCT]:   Writing Kinetics in File: ' + KineticFile )
        csvkinetics  = open(KineticFile, 'w')

    elif (iFlg == -2):
        print('[SurQCT]:   Closing Kinetics File: ' + KineticFile )
        csvkinetics.close()


    return csvkinetics



In [36]:
if(ReactionType == 'KInel'): 
    PathToKineticFile  = FileName
    Rates, i2j_dict, iIdx, jIdx, ARate, BRate, CRate = read_kexcitdata_PLATO(PathToKineticFile)
elif(ReactionType == 'KExch'): 
    PathToKineticFile  = FileName
    Rates, i2j_dict, iIdx, jIdx, ARate, BRate, CRate = read_kexcitdata_PLATO(PathToKineticFile)
elif(ReactionType == 'KDiss'): 
    PathToKineticFile  = FileName
    Rates, i2j_dict, iIdx, jIdx, ARate, BRate, CRate = read_kdissdata_PLATO(PathToKineticFile)


/home/venturi/WORKSPACE//../N2_sanjeev.dat
N2,0,,=N2,1,,:1.22575292e-02,2.37427314e+00,1.06996382e+04,6                                      



IndexError: list index out of range

In [28]:
float("2.37427314e-04")

0.000237427314